In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import rc # 한글 그래픽 처리
import seaborn as sns # r에서 ggplot이라 생각하면됨. 쓸때 한글처리 주의.
%matplotlib inline
rc('font', family='malgun gothic')
rc('axes', unicode_minus=False)

In [2]:
train = pd.read_csv('./train.csv',engine='python',encoding='cp949')
a = []
for i in range(len(train)):
    a.append(int(train['gdp_for_year'][i].replace(',','')))
train['gdp_for_year'] = a
train.head(3)

,country,year,sex,age,suicides_no,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,gdp_per_capital,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,2156624900,796,Generation X


In [3]:
tmp = pd.read_csv("./test_mini.csv")
a = []
for i in range(len(tmp)):
    a.append(int(tmp['gdp_for_year'][i].replace(',','')))
tmp['gdp_for_year'] = a
tmp.head()

,country,year,sex,age,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,gdp_per_capital,generation
0,Antigua and Barbuda,2015,female,55-74 years,6403,15.62,Antigua and Barbuda2015,NaN,1364863037,14853,Boomers
1,Antigua and Barbuda,2015,female,15-24 years,8561,0.00,Antigua and Barbuda2015,NaN,1364863037,14853,Millenials
2,Antigua and Barbuda,2015,female,25-34 years,7740,0.00,Antigua and Barbuda2015,NaN,1364863037,14853,Millenials
3,Antigua and Barbuda,2015,female,35-54 years,15323,0.00,Antigua and Barbuda2015,NaN,1364863037,14853,Generation X
4,Antigua and Barbuda,2015,female,5-14 years,8239,0.00,Antigua and Barbuda2015,NaN,1364863037,14853,Generation Z


In [4]:
gold = pd.read_csv('./국제금값종장기준(일간).csv',encoding='cp949',engine='python')
oilUSA = pd.read_csv('./텍사스산기름값데이터.csv',encoding='cp949',engine='python')
product = pd.read_csv('./모든상품의글로벌가격지수.csv',encoding='cp949',engine='python')
oilEU = pd.read_csv('./유럽산기름값연간데이터.csv',encoding='cp949',engine='python')

In [5]:
gold.head(3), oilUSA.head(3), product.head(3), oilEU.head(3)

(         DATE   PRICE
 0  1987-01-02  403.50
 1  1987-01-05  398.95
 2  1987-01-06  401.70,          DATE  TEXASprice
 0  1986-01-01       15.05
 1  1987-01-01       19.20
 2  1988-01-01       15.97,          DATE  productprice
 0  1992-01-01         55.04
 1  1993-01-01         52.68
 2  1994-01-01         54.92,          DATE  Euprice
 0  1987-01-01    18.53
 1  1988-01-01    14.91
 2  1989-01-01    18.23)

In [6]:
gold.DATE = pd.to_datetime(gold.DATE,format = '%Y-%m-%d')
gold['DATE'] = gold.DATE.dt.year
gold_data = gold.groupby('DATE')['PRICE'].agg([('goldmin',min),('goldmax',max),('goldmean','mean')]).reset_index()
gold_data['minmaxprice'] = abs(gold_data.goldmax - gold_data.goldmin)
gold_data.head(2)

,DATE,goldmin,goldmax,goldmean,minmaxprice
0,1987,390.0,499.75,446.223705,109.75
1,1988,395.3,483.90,436.858167,88.60


In [7]:
oilUSA.DATE = pd.to_datetime(oilUSA.DATE,format = '%Y-%m-%d')
oilUSA['DATE'] = oilUSA.DATE.dt.year
product.DATE = pd.to_datetime(product.DATE,format = '%Y-%m-%d')
product['DATE'] = product.DATE.dt.year
oilEU.DATE = pd.to_datetime(oilEU.DATE,format = '%Y-%m-%d')
oilEU['DATE'] = oilEU.DATE.dt.year

print(oilUSA.head(2))
print(product.head(2))
print(oilEU.head(2))

   DATE  TEXASprice
0  1986       15.05
1  1987       19.20
   DATE  productprice
0  1992         55.04
1  1993         52.68
   DATE  Euprice
0  1987    18.53
1  1988    14.91


In [8]:
addDATA = pd.merge(product,pd.merge(oilEU,pd.merge(gold_data,oilUSA,on='DATE'),on='DATE'),on='DATE')
addDATA = addDATA.rename(columns={'DATE':'year'})
addDATA.head(2)

,year,productprice,Euprice,goldmin,goldmax,goldmean,minmaxprice,TEXASprice
0,1992,55.04,19.32,330.35,359.6,343.949206,29.25,20.58
1,1993,52.68,17.01,326.10,405.6,359.815139,79.50,18.43


In [9]:
country = tmp.country.unique()

addtrain = pd.merge(train,addDATA,on='year')
addtrain = addtrain.query("country in @country")
addtrain.head(3)

,country,year,sex,age,suicides_no,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,gdp_per_capital,generation,productprice,Euprice,goldmin,goldmax,goldmean,minmaxprice,TEXASprice
12,Antigua and Barbuda,1992,female,15-24 years,0,6369,0.0,Antigua and Barbuda1992,NaN,499281148,8087,Generation X,55.04,19.32,330.35,359.6,343.949206,29.25,20.58
13,Antigua and Barbuda,1992,female,25-34 years,0,6650,0.0,Antigua and Barbuda1992,NaN,499281148,8087,Boomers,55.04,19.32,330.35,359.6,343.949206,29.25,20.58
14,Antigua and Barbuda,1992,female,35-54 years,0,6534,0.0,Antigua and Barbuda1992,NaN,499281148,8087,Boomers,55.04,19.32,330.35,359.6,343.949206,29.25,20.58


In [10]:
addtest = pd.merge(tmp,addDATA,on='year')
print(addtest.head(3))
print(addtest.tail(3))

               country  year     sex          age  population  \
0  Antigua and Barbuda  2015  female  55-74 years        6403   
1  Antigua and Barbuda  2015  female  15-24 years        8561   
2  Antigua and Barbuda  2015  female  25-34 years        7740   

   suicides_100k_pop             country_year  HDI_for_year  gdp_for_year  \
0              15.62  Antigua and Barbuda2015           NaN    1364863037   
1               0.00  Antigua and Barbuda2015           NaN    1364863037   
2               0.00  Antigua and Barbuda2015           NaN    1364863037   

   gdp_per_capital  generation  productprice  Euprice  goldmin  goldmax  \
0            14853     Boomers        111.26    52.32   1049.4  1295.75   
1            14853  Millenials        111.26    52.32   1049.4  1295.75   
2            14853  Millenials        111.26    52.32   1049.4  1295.75   

      goldmean  minmaxprice  TEXASprice  
0  1160.062351       246.35       48.66  
1  1160.062351       246.35       48.66  
2  

In [11]:
unemploy_m = pd.read_table('./Unemployment_male (%).csv', encoding='euc-kr')
cols = list(range(1, 27))
unemploy_m.drop(unemploy_m.columns[cols], axis=1, inplace=True)
unemploy_m.drop(['2016', '2017', '2018'], axis=1, inplace=True)

unemploy_m_melt = pd.melt(unemploy_m.iloc[:,1:])
unemploy_m_melt.columns = ['year', 'unemployment rate']
unemploy_m_CountryName = list(unemploy_m['Country Name'])*31
unemploy_m_CountryName = pd.DataFrame({'Country Name' : unemploy_m_CountryName})
unemploy_m = pd.concat([unemploy_m_CountryName, unemploy_m_melt], axis=1)
unemploy_m['sex']='male'
unemploy_m = unemploy_m.iloc[:,[0, 1, 3, 2]]
unemploy_m.head(2)

/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,Country Name,year,sex,unemployment rate
0,Aruba,1985,male,NaN
1,Afghanistan,1985,male,NaN


In [12]:
unemploy_f = pd.read_table('./Unemployment_female (%).csv', encoding='euc-kr')
cols = list(range(1, 26))
unemploy_f.drop(unemploy_f.columns[cols], axis=1, inplace=True)
unemploy_f.drop(['2016', '2017', '2018'], axis=1, inplace=True)

unemploy_f_melt = pd.melt(unemploy_f.iloc[:,1:])
unemploy_f_melt.columns = ['year', 'unemployment rate']
unemploy_f_CountryName = list(unemploy_f['Country Name'])*31
unemploy_f_CountryName = pd.DataFrame({'Country Name' : unemploy_f_CountryName})
unemploy_f = pd.concat([unemploy_f_CountryName, unemploy_f_melt], axis=1)
unemploy_f['sex']='female'
unemploy_f = unemploy_f.iloc[:,[0, 1, 3, 2]]
unemploy_f.head(2)

/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,Country Name,year,sex,unemployment rate
0,Aruba,1985,female,NaN
1,Afghanistan,1985,female,NaN


In [13]:
unemploy = pd.concat([unemploy_f, unemploy_m])
unemploy = unemploy.sort_values(by=['Country Name', 'year'], ascending=True)
unemploy.head(3)

,Country Name,year,sex,unemployment rate
1,Afghanistan,1985,female,NaN
1,Afghanistan,1985,male,NaN
265,Afghanistan,1986,female,NaN


In [14]:
unemploy = unemploy[(unemploy.year!='1985') & (unemploy.year!='1986') & (unemploy.year!='1987') & (unemploy.year!='1988') & 
                    (unemploy.year!='1989') & (unemploy.year!='1990')]
unemploy.head()
## 이것을 하는 이유는 unemploy데이터 내 1985년~1990년은 실업률 데이터가 전부 NaN이기 때문!!

,Country Name,year,sex,unemployment rate
1585,Afghanistan,1991,female,14.561
1585,Afghanistan,1991,male,11.616
1849,Afghanistan,1992,female,14.068
1849,Afghanistan,1992,male,10.085
2113,Afghanistan,1993,female,14.702


In [15]:
unemploy['Country Name'].replace(to_replace=['Korea, Rep.'], value='Republic of Korea', inplace=True)
unemploy['Country Name'].replace(to_replace=['Slovak Republic'], value='Slovakia', inplace=True)
unemploy['Country Name'].replace(to_replace=['Kyrgyz Republic'], value='Kyrgyzstan', inplace=True)
unemploy['Country Name'].replace(to_replace=['Macao SAR, China'], value='Macau', inplace=True)
unemploy['Country Name'].replace(to_replace=['Bahamas, The'], value='Bahamas', inplace=True)
unemploy['Country Name'].replace(to_replace=['St. Kitts and Nevis'], value='Saint Kitts and Nevis', inplace=True)
unemploy['Country Name'].replace(to_replace=['St. Lucia'], value='Saint Lucia', inplace=True)
unemploy['Country Name'].replace(to_replace=['St. Vincent and the Grenadines'], value='Saint Vincent and Grenadines', inplace=True)

In [16]:
unemploy['year'] = list(map(int, unemploy['year']))
unemploy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13200 entries, 1585 to 8183
Data columns (total 4 columns):
Country Name         13200 non-null object
year                 13200 non-null int64
sex                  13200 non-null object
unemployment rate    11650 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 515.6+ KB


In [17]:
addtrain.head(2)

,country,year,sex,age,suicides_no,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,gdp_per_capital,generation,productprice,Euprice,goldmin,goldmax,goldmean,minmaxprice,TEXASprice
12,Antigua and Barbuda,1992,female,15-24 years,0,6369,0.0,Antigua and Barbuda1992,NaN,499281148,8087,Generation X,55.04,19.32,330.35,359.6,343.949206,29.25,20.58
13,Antigua and Barbuda,1992,female,25-34 years,0,6650,0.0,Antigua and Barbuda1992,NaN,499281148,8087,Boomers,55.04,19.32,330.35,359.6,343.949206,29.25,20.58


In [18]:
addtrain = pd.merge(addtrain, unemploy, left_on=['country', 'year', 'sex'], 
                   right_on=['Country Name', 'year', 'sex'], 
                   how='left').drop(['Country Name'], axis=1)
addtest = pd.merge(addtest, unemploy, left_on=['country', 'year', 'sex'], 
                   right_on=['Country Name', 'year', 'sex'], 
                   how='left').drop(['Country Name'], axis=1)

In [19]:
prevalence_depress = pd.read_csv('./share-with-depression-by-gdp-pc.csv')
prevalence_depress = prevalence_depress.iloc[:,:4]
prevalence_depress.drop('Code', axis=1, inplace=True)
prevalence_depress.columns=['country', 'year', 'prevalence of depressive disorders (%)']
prevalence_depress.head(3)

prevalence_depress['country'].replace(to_replace=['Cape Verde'], value='Cabo Verde', inplace=True)
prevalence_depress['country'].replace(to_replace=['Macao'], value='Macau', inplace=True)
prevalence_depress['country'].replace(to_replace=['South Korea'], value='Republic of Korea', inplace=True)
prevalence_depress['country'].replace(to_replace=['Russia'], value='Russian Federation', inplace=True)
prevalence_depress['country'].replace(to_replace=['Saint Vincent and the Grenadines'], 
                                      value='Saint Vincent and Grenadines', inplace=True)


In [20]:
addtrain = pd.merge(addtrain, prevalence_depress, on=['country', 'year'], how='left')
addtest = pd.merge(addtest, prevalence_depress, on=['country', 'year'], how='left')


In [21]:
addtest.head(3)

,country,year,sex,age,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,gdp_per_capital,generation,productprice,Euprice,goldmin,goldmax,goldmean,minmaxprice,TEXASprice,unemployment rate,prevalence of depressive disorders (%)
0,Antigua and Barbuda,2015,female,55-74 years,6403,15.62,Antigua and Barbuda2015,NaN,1364863037,14853,Boomers,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,NaN,3.122445
1,Antigua and Barbuda,2015,female,15-24 years,8561,0.00,Antigua and Barbuda2015,NaN,1364863037,14853,Millenials,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,NaN,3.122445
2,Antigua and Barbuda,2015,female,25-34 years,7740,0.00,Antigua and Barbuda2015,NaN,1364863037,14853,Millenials,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,NaN,3.122445


In [22]:
addtest.drop('HDI_for_year', axis=1, inplace=True)
addtrain.drop('HDI_for_year', axis=1, inplace=True)

addtest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 0 to 743
Data columns (total 19 columns):
country                                   744 non-null object
year                                      744 non-null int64
sex                                       744 non-null object
age                                       744 non-null object
population                                744 non-null int64
suicides_100k_pop                         744 non-null float64
country_year                              744 non-null object
gdp_for_year                              744 non-null int64
gdp_per_capital                           744 non-null int64
generation                                744 non-null object
productprice                              744 non-null float64
Euprice                                   744 non-null float64
goldmin                                   744 non-null float64
goldmax                                   744 non-null float64
goldmean                  

In [23]:
addtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15624 entries, 0 to 15623
Data columns (total 20 columns):
country                                   15624 non-null object
year                                      15624 non-null int64
sex                                       15624 non-null object
age                                       15624 non-null object
suicides_no                               15624 non-null int64
population                                15624 non-null int64
suicides_100k_pop                         15624 non-null float64
country_year                              15624 non-null object
gdp_for_year                              15624 non-null int64
gdp_per_capital                           15624 non-null int64
generation                                15624 non-null object
productprice                              15624 non-null float64
Euprice                                   15624 non-null float64
goldmin                                   15624 non-null floa

In [24]:
groupdata = addtrain.groupby(['country','year', 'sex']).agg({'suicides_no':'sum','population':'sum','gdp_for_year':'mean',
                                                             'gdp_per_capital':'mean','productprice':'mean','Euprice':'mean',
                                                             'goldmin':'mean','goldmax':'mean','goldmean':'mean','minmaxprice':'mean',
                                                             'TEXASprice':'mean', 'unemployment rate':'mean', 
                                                             'prevalence of depressive disorders (%)':'mean'}).reset_index()
grouptest = addtest.groupby(['country','year', 'sex']).agg({'population':'sum','gdp_for_year':'mean',
                                                            'gdp_per_capital':'mean','productprice':'mean','Euprice':'mean',
                                                            'goldmin':'mean','goldmax':'mean','goldmean':'mean','minmaxprice':'mean',
                                                            'TEXASprice':'mean', 'unemployment rate':'mean', 
                                                            'prevalence of depressive disorders (%)':'mean'}).reset_index()

In [25]:
groupdata['1인당GDP'] = groupdata['gdp_for_year']/groupdata['population']*100
groupdata['oilprice'] = groupdata['Euprice']/groupdata['TEXASprice']*100
groupdata['gdp_product'] = groupdata['gdp_for_year']/groupdata['productprice']*100
groupdata['gdp_gold'] = groupdata['gdp_for_year']/groupdata['goldmean']*100
grouptest['1인당GDP'] = grouptest['gdp_for_year']/grouptest['population']*100
grouptest['oilprice'] = grouptest['Euprice']/grouptest['TEXASprice']*100
grouptest['gdp_product'] = grouptest['gdp_for_year']/grouptest['productprice']*100
grouptest['gdp_gold'] = grouptest['gdp_for_year']/grouptest['goldmean']*100

In [26]:
abs(groupdata.corr()['suicides_no']).sort_values(ascending=False)

suicides_no                               1.000000
population                                0.645288
gdp_for_year                              0.528072
gdp_product                               0.503755
gdp_gold                                  0.481418
prevalence of depressive disorders (%)    0.166813
unemployment rate                         0.094387
gdp_per_capital                           0.033159
1인당GDP                                    0.032113
year                                      0.028569
TEXASprice                                0.026749
Euprice                                   0.026433
productprice                              0.026155
goldmax                                   0.023836
goldmean                                  0.023523
goldmin                                   0.023225
minmaxprice                               0.022488
oilprice                                  0.022036
Name: suicides_no, dtype: float64

In [27]:
### sex : female은 0으로, male은 1로 인코딩하기
groupdata['sex'] = groupdata['sex'].map({'female':0, 'male':1})
grouptest['sex'] = grouptest['sex'].map({'female':0, 'male':1})

print(groupdata.head(2))
print(grouptest.head(2))

               country  year  sex  suicides_no  population  gdp_for_year  \
0  Antigua and Barbuda  1992    0            0       31851     499281148   
1  Antigua and Barbuda  1992    1            0       29890     499281148   

   gdp_per_capital  productprice  Euprice  goldmin  goldmax    goldmean  \
0             8087         55.04    19.32   330.35    359.6  343.949206   
1             8087         55.04    19.32   330.35    359.6  343.949206   

   minmaxprice  TEXASprice  unemployment rate  \
0        29.25       20.58                NaN   
1        29.25       20.58                NaN   

   prevalence of depressive disorders (%)        1인당GDP   oilprice  \
0                                3.147342  1.567553e+06  93.877551   
1                                3.147342  1.670395e+06  93.877551   

    gdp_product      gdp_gold  
0  9.071242e+08  1.451613e+08  
1  9.071242e+08  1.451613e+08  
               country  year  sex  population  gdp_for_year  gdp_per_capital  \
0  Antigua

In [28]:
groupdata.info(), grouptest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2604 entries, 0 to 2603
Data columns (total 20 columns):
country                                   2604 non-null object
year                                      2604 non-null int64
sex                                       2604 non-null int64
suicides_no                               2604 non-null int64
population                                2604 non-null int64
gdp_for_year                              2604 non-null int64
gdp_per_capital                           2604 non-null int64
productprice                              2604 non-null float64
Euprice                                   2604 non-null float64
goldmin                                   2604 non-null float64
goldmax                                   2604 non-null float64
goldmean                                  2604 non-null float64
minmaxprice                               2604 non-null float64
TEXASprice                                2604 non-null float64
unemployme

(None, None)

In [29]:
groupdata['unemployment rate'] = groupdata['unemployment rate'].fillna(np.mean(groupdata['unemployment rate']))
groupdata['prevalence of depressive disorders (%)'] = groupdata['prevalence of depressive disorders (%)'].fillna(np.mean(groupdata['prevalence of depressive disorders (%)']))

In [30]:
grouptest['unemployment rate'] = grouptest['unemployment rate'].fillna(np.mean(grouptest['unemployment rate']))
grouptest['prevalence of depressive disorders (%)'] = grouptest['prevalence of depressive disorders (%)'].fillna(np.mean(grouptest['prevalence of depressive disorders (%)']))

In [31]:
groupdata.head(10)

,country,year,sex,suicides_no,population,gdp_for_year,gdp_per_capital,productprice,Euprice,goldmin,goldmax,goldmean,minmaxprice,TEXASprice,unemployment rate,prevalence of depressive disorders (%),1인당GDP,oilprice,gdp_product,gdp_gold
0,Antigua and Barbuda,1992,0,0,31851,499281148,8087,55.04,19.32,330.35,359.60,343.949206,29.25,20.58,8.652178,3.147342,1.567553e+06,93.877551,9.071242e+08,1.451613e+08
1,Antigua and Barbuda,1992,1,0,29890,499281148,8087,55.04,19.32,330.35,359.60,343.949206,29.25,20.58,8.652178,3.147342,1.670395e+06,93.877551,9.071242e+08,1.451613e+08
2,Antigua and Barbuda,1993,0,0,32590,535172778,8453,52.68,17.01,326.10,405.60,359.815139,79.50,18.43,8.652178,3.132672,1.642138e+06,92.295171,1.015894e+09,1.487355e+08
3,Antigua and Barbuda,1993,1,1,30725,535172778,8453,52.68,17.01,326.10,405.60,359.815139,79.50,18.43,8.652178,3.132672,1.741815e+06,92.295171,1.015894e+09,1.487355e+08
4,Antigua and Barbuda,1994,0,0,33465,589429593,9062,54.92,15.86,369.65,396.25,384.151400,26.60,17.20,8.652178,3.115920,1.761332e+06,92.209302,1.073251e+09,1.534368e+08
5,Antigua and Barbuda,1994,1,0,31579,589429593,9062,54.92,15.86,369.65,396.25,384.151400,26.60,17.20,8.652178,3.115920,1.866524e+06,92.209302,1.073251e+09,1.534368e+08
6,Antigua and Barbuda,1995,0,0,34464,577280741,8636,59.40,17.02,372.40,395.55,384.053200,23.15,18.43,8.652178,3.109716,1.675025e+06,92.349430,9.718531e+08,1.503127e+08
7,Antigua and Barbuda,1995,1,0,32382,577280741,8636,59.40,17.02,372.40,395.55,384.053200,23.15,18.43,8.652178,3.109716,1.782721e+06,92.349430,9.718531e+08,1.503127e+08
8,Antigua and Barbuda,1998,0,0,38056,727860593,10078,47.72,12.76,273.40,313.15,294.087052,39.75,14.42,8.652178,3.126194,1.912604e+06,88.488211,1.525274e+09,2.474983e+08
9,Antigua and Barbuda,1998,1,0,34169,727860593,10078,47.72,12.76,273.40,313.15,294.087052,39.75,14.42,8.652178,3.126194,2.130178e+06,88.488211,1.525274e+09,2.474983e+08


In [32]:
len(groupdata.columns)

20

In [33]:
grouptest.head(10)

,country,year,sex,population,gdp_for_year,gdp_per_capital,productprice,Euprice,goldmin,goldmax,goldmean,minmaxprice,TEXASprice,unemployment rate,prevalence of depressive disorders (%),1인당GDP,oilprice,gdp_product,gdp_gold
0,Antigua and Barbuda,2015,0,47990,1364863037,14853,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,8.184898,3.122445,2.844057e+06,107.521578,1.226733e+09,1.176543e+08
1,Antigua and Barbuda,2015,1,43899,1364863037,14853,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,8.184898,3.122445,3.109098e+06,107.521578,1.226733e+09,1.176543e+08
2,Argentina,2015,0,20346129,594749285413,14981,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,8.234000,3.833964,2.923157e+06,107.521578,5.345580e+11,5.126873e+10
3,Argentina,2015,1,19353495,594749285413,14981,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,6.401000,3.833964,3.073085e+06,107.521578,5.345580e+11,5.126873e+10
4,Armenia,2015,0,1472585,10553337673,3775,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,19.257999,3.057042,7.166539e+05,107.521578,9.485294e+09,9.097216e+08
5,Armenia,2015,1,1322750,10553337673,3775,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,17.385000,3.057042,7.978331e+05,107.521578,9.485294e+09,9.097216e+08
6,Australia,2015,0,11201512,1349034029453,60656,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,6.071000,4.974322,1.204332e+07,107.521578,1.212506e+12,1.162898e+11
7,Australia,2015,1,11039273,1349034029453,60656,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,6.051000,4.974322,1.222032e+07,107.521578,1.212506e+12,1.162898e+11
8,Austria,2015,0,4201798,382065930308,46484,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,5.304000,3.487289,9.092915e+06,107.521578,3.433992e+11,3.293495e+10
9,Austria,2015,1,4017588,382065930308,46484,111.26,52.32,1049.4,1295.75,1160.062351,246.35,48.66,6.084000,3.487289,9.509834e+06,107.521578,3.433992e+11,3.293495e+10


### RandomForestRegressor를 썼을 때(국가와 성별을 기준으로 예측함) → RMSE=760

In [34]:
###### 국가별 성별로 
country = tmp.country.unique()

from sklearn.ensemble import RandomForestRegressor

nation = []
predict = []

for i in country:
    lastdata = groupdata.query("country == @i").drop(['country'],axis=1)
    testdata = grouptest.query("country == @i").drop(['country'],axis=1)
    X = lastdata.drop('suicides_no',axis=1)
    y = lastdata['suicides_no']
    #X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
    #model = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
    # model = RandomForestRegressor(random_state=42, n_estimators=100).fit(X, y)
    model = RandomForestRegressor(random_state=42, n_estimators=100).fit(X, y)
    #modelaccuracy.append(model.score(X_test, y_test))
    predict.append(model.predict(testdata)[0])
    predict.append(model.predict(testdata)[1])
    nation.append(i)

In [35]:
len(nation), len(predict)

(62, 124)

In [36]:
nation1=[]
for i in nation:
    nation1 +=[i for n in range(2)]
nation1[:11]
print(len(nation1))

124


In [37]:
pre = pd.DataFrame({'country':nation1,'predict':predict, 'sex':[0, 1]*62})#,'accuracy':modelaccuracy})
#DATA = pre.groupby('country')['predict'].sum().reset_index()
result = pd.read_csv("who_suicide_statistics.csv")
result.head()

,country,year,sex,age,suicides_no,population
0,Albania,1985,female,15-24 years,NaN,277900.0
1,Albania,1985,female,25-34 years,NaN,246800.0
2,Albania,1985,female,35-54 years,NaN,267500.0
3,Albania,1985,female,5-14 years,NaN,298300.0
4,Albania,1985,female,55-74 years,NaN,138700.0


In [38]:
result['sex']=result['sex'].map({'female':0, 'male':1})
result.head()

,country,year,sex,age,suicides_no,population
0,Albania,1985,0,15-24 years,NaN,277900.0
1,Albania,1985,0,25-34 years,NaN,246800.0
2,Albania,1985,0,35-54 years,NaN,267500.0
3,Albania,1985,0,5-14 years,NaN,298300.0
4,Albania,1985,0,55-74 years,NaN,138700.0


In [39]:
compare = result.query('year == 2015').groupby(['country', 'sex']).agg({'suicides_no':'sum'}).reset_index()
fin = pd.merge(pre, compare, on=['country', 'sex'])

In [40]:
y_true = fin.suicides_no
y_pred = fin.predict
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_true, y_pred))

760.1264368646125

In [41]:
predict_df = pd.DataFrame({'country':fin.country, 'sex':[0,1]*61, 
                          'true':fin.suicides_no, 'pred':fin.predict})
predict_df

,country,sex,true,pred
0,Antigua and Barbuda,0,1.0,0.15
1,Antigua and Barbuda,1,0.0,0.20
2,Argentina,0,604.0,640.38
3,Argentina,1,2469.0,2466.15
4,Armenia,0,19.0,14.50
5,Armenia,1,55.0,47.51
6,Australia,0,735.0,629.38
7,Australia,1,2292.0,2077.41
8,Austria,0,290.0,316.36
9,Austria,1,961.0,992.00


### Decition Tree를 썼을 때(국가와 성별을 기준으로 예측함) → RMSE = 196.7

In [42]:
from sklearn.tree import DecisionTreeRegressor

nation2 = []
predict1 = []

for i in country:
    lastdata = groupdata.query("country == @i").drop(['country'],axis=1)
    testdata = grouptest.query("country == @i").drop(['country'],axis=1)
    X = lastdata.drop('suicides_no',axis=1)
    y = lastdata['suicides_no']
    #X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
    #model = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
    # model = DecisionTreeRegressor(max_depth=8,random_state=42).fit(X, y)
    model = DecisionTreeRegressor(max_depth=8,random_state=42).fit(X, y)
    #modelaccuracy.append(model.score(X_test, y_test))
    predict1.append(model.predict(testdata)[0])
    predict1.append(model.predict(testdata)[1])
    nation2.append(i)
    
print(len(nation2), len(predict1))

62 124


In [43]:
nation3=[]
for i in nation2:
    nation3 +=[i for n in range(2)]
nation3[:11]
print(len(nation3))

pre1 = pd.DataFrame({'country':nation3,'predict1':predict1, 'sex':[0, 1]*62})

result1 = pd.read_csv("./who_suicide_statistics.csv")
result1['sex']=result1['sex'].map({'female':0, 'male':1})

compare1 = result1.query('year == 2015').groupby(['country', 'sex']).agg({'suicides_no':'sum'}).reset_index()
fin1 = pd.merge(pre1,compare1,on=['country', 'sex'])
y_true1 = fin1.suicides_no
y_pred1 = fin1.predict1
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_true1, y_pred1))

124


196.70144567792343

In [44]:
len(fin.suicides_no), len(fin.predict), len(fin1.predict1),

(122, 122, 122)

In [45]:
predict_df = pd.DataFrame({'country':fin.country, 'sex':[0,1]*61, 
                          'true':fin.suicides_no, 'pred_using_RandomForest':fin.predict, 'pred_using_DecisionTree':fin1.predict1})
predict_df

,country,sex,true,pred_using_RandomForest,pred_using_DecisionTree
0,Antigua and Barbuda,0,1.0,0.15,0.00
1,Antigua and Barbuda,1,0.0,0.20,0.00
2,Argentina,0,604.0,640.38,687.00
3,Argentina,1,2469.0,2466.15,2544.00
4,Armenia,0,19.0,14.50,10.00
5,Armenia,1,55.0,47.51,59.00
6,Australia,0,735.0,629.38,709.00
7,Australia,1,2292.0,2077.41,2182.00
8,Austria,0,290.0,316.36,324.00
9,Austria,1,961.0,992.00,990.00


In [46]:
predict_df['sex']=predict_df['sex'].map({0:'female', 1:'male'})
predict_df

,country,sex,true,pred_using_RandomForest,pred_using_DecisionTree
0,Antigua and Barbuda,female,1.0,0.15,0.00
1,Antigua and Barbuda,male,0.0,0.20,0.00
2,Argentina,female,604.0,640.38,687.00
3,Argentina,male,2469.0,2466.15,2544.00
4,Armenia,female,19.0,14.50,10.00
5,Armenia,male,55.0,47.51,59.00
6,Australia,female,735.0,629.38,709.00
7,Australia,male,2292.0,2077.41,2182.00
8,Austria,female,290.0,316.36,324.00
9,Austria,male,961.0,992.00,990.00


In [51]:
predict_df.groupby('country').pred_using_DecisionTree.sum().reset_index().to_csv("현정.csv")